# Variables

## Import des librairies

In [ ]:
import os
import sys
import csv
import json
from rdflib import Graph, Namespace, URIRef, Literal, BNode, XSD

## Définition des variables globales

In [ ]:
# Fichiers existants
ont_file_name = "ontology.ttl" # Fusion des deux fichiers précédents
ruleset_file_name = "rules.pie"

# Fichiers créés durant le processus
export_file_name = "addresses-temp.ttl"
out_file_name = "addresses.ttl"
local_config_file_name = "config_repo.ttl"
facts_ttl_file_name = "facts_data.ttl"

# Dossiers existants
data_folder_name = "data"
mapping_folder_name = "mappings"

# Dossiers créés durant le traitement
tmp_folder_name = "tmp_files"

# Nom du répertoire GraphDB pour les faits
facts_repository_name = "addresses_from_factoids"

# Définition des graphes nommés
ontology_named_graph_name = "ontology"
facts_graph_name = "facts"
factoids_graph_name = "factoids"
permanent_graph_name = "permanent"

# Définition de namespaces et de préfixes associés
namespace_prefixes = {"addr":Namespace("http://rdf.geohistoricaldata.org/def/address#"),
                      "facts":Namespace("http://rdf.geohistoricaldata.org/id/address/facts/"),
                      "factoids":Namespace("http://rdf.geohistoricaldata.org/id/address/factoids/"),
                     }

# URIs pour accéder aux logiciels GraphDB et Ontotext-Refine
graphdb_url = "http://localhost:7200"
ontorefine_url = "http://localhost:7333"

# Commande pour lancer `Ontorefine`, dépend de l'OS et d'où il se situe
# ontorefine_cmd = "ontorefine-cli" # Nom sans chemin
ontorefine_cmd = "/opt/ontotext-refine/lib/app/bin/ontorefine-cli" #Ubuntu
# ontorefine_cmd = "/Applications/Ontotext\\ Refine.app/Contents/app/bin/ontorefine-cli" #MacOS

py_code_folder_path = "../code"

## Traitement des variables globales

* Obtention des chemins absolus des fichiers à partir des chemins relatifs donnés dans la section précédente
* Création d'un dossier temporaire s'il n'existe pas encore pour stocker des fichiers à vocation d'être supprimés

In [ ]:
tmp_folder = os.path.abspath(tmp_folder_name)
mapping_folder = os.path.abspath(mapping_folder_name)
data_folder = os.path.abspath(data_folder_name)

python_code_folder = os.path.abspath(py_code_folder_path)

local_config_file = os.path.join(tmp_folder, local_config_file_name)
ont_file = os.path.abspath(ont_file_name)
ruleset_file = os.path.abspath(ruleset_file_name)
facts_ttl_file = os.path.join(tmp_folder, facts_ttl_file_name)

## Import des modules situés dans le dossier `code`

In [ ]:
# Appel du dossier `code` comprend les codes python
sys.path.insert(1, python_code_folder)

import filemanagement as fm
import wikidata as wd
import ontorefine as otr
import graphdb as gd
import graphrdf as gr
import strprocessing as sp
import geomprocessing as gp

## Création de dossiers s'ils n'existent pas

In [ ]:
fm.create_folder_if_not_exists(tmp_folder)

## Gestion du répertoire local

### Création du répertoire local dans GraphDB
Pour que la création marche, il faut que GraphDB soit lancé et donc que l'URI donné par `graphdb_url` fonctionne. Si le répertoire existe déjà, rien n'est fait

In [ ]:
# gd.remove_repository(graphdb_url, facts_repository_name)
gd.create_config_local_repository_file(local_config_file, facts_repository_name, ruleset_file=ruleset_file, disable_same_as=False)
gd.create_repository_from_config_file(graphdb_url, local_config_file)

### Vidage du répertoire local
Le répertoire dont l'id est `repository_name` pour y stocker les données récupérées, cela est utile si le répertoire existait déjà.

In [ ]:
gd.clear_repository(graphdb_url, facts_repository_name)

## Import de l'ontologie

In [ ]:
gd.import_ttl_file_in_graphdb(graphdb_url, facts_repository_name, ont_file, ontology_named_graph_name)

## Définition de variables liées aux sources

### Voies de Paris via Wikidata

* `wd` pour "wikidata"
* `wdpt` pour "wikidata paris thoroughfares"
* `wdpa` pour "wikidata paris areas"
* `wdpl` pour "wikidata paris thoroughfare locations"

In [ ]:
# Nom du répertoire où sont stockés et construits les triplets des factoïdes des données de Wikidata
wd_repository_name = "factoids_wikidata"

# Fichier CSV pour stocker le résultat de la requête de la sélection
wdpt_csv_file_name = "wd_paris_thoroughfares.csv"
wdpt_csv_file = os.path.join(data_folder, wdpt_csv_file_name)

# Fichier de mapping pour transformer le fichier CSV en fichier TTL
wdpt_mapping_file_name = "mapping_wikidata_landmarks_thoroughfares.json"
wdpt_mapping_file = os.path.join(mapping_folder, wdpt_mapping_file_name)

# Fichier TTL pour structurer les connaissances des voies de Paris
wdpt_kg_file_name = "wd_paris_thoroughfares.ttl"
wdpt_kg_file = os.path.join(tmp_folder, wdpt_kg_file_name)

# Fichier CSV pour stocker le résultat de la requête de la sélection
wdpa_csv_file_name = "wd_paris_areas.csv"
wdpa_csv_file = os.path.join(data_folder, wdpa_csv_file_name)

# Fichier de mapping pour transformer le fichier CSV en fichier TTL
wdpa_mapping_file_name = "mapping_wikidata_landmarks_areas.json"
wdpa_mapping_file = os.path.join(mapping_folder, wdpa_mapping_file_name)

# Fichier TTL pour structurer les connaissances des zones de Paris
wdpa_kg_file_name = "wd_paris_areas.ttl"
wdpa_kg_file = os.path.join(tmp_folder, wdpa_kg_file_name)

# Fichier CSV pour stocker le résultat de la requête de la sélection
wdpl_csv_file_name = "wd_paris_locations.csv"
wdpl_csv_file = os.path.join(data_folder, wdpl_csv_file_name)

# Fichier de mapping pour transformer le fichier CSV en fichier TTL
wdpl_mapping_file_name = "mapping_wikidata_landmark_relations.json"
wdpl_mapping_file = os.path.join(mapping_folder, wdpl_mapping_file_name)

# Fichier TTL pour structurer les connaissances des zones de Paris
wdpl_kg_file_name = "wd_paris_thoroughfare_locations.ttl"
wdpl_kg_file = os.path.join(tmp_folder, wdpl_kg_file_name)

# Fichier TTL final des factoïdes de Wikidata
wd_factoids_kg_file_name = "wd_factoids.ttl"
wd_factoids_kg_file = os.path.join(tmp_folder, wd_factoids_kg_file_name)
wd_permanent_kg_file_name = "wd_permanent.ttl"
wd_permanent_kg_file = os.path.join(tmp_folder, wd_permanent_kg_file_name)

### Nomenclature des voies de la ville de Paris

Les données de la ville de Paris sont composées de deux jeux :
* [dénominations des emprises des voies actuelles](https://opendata.paris.fr/explore/dataset/denominations-emprises-voies-actuelles)
* [dénominations caduques des voies](https://opendata.paris.fr/explore/dataset/denominations-des-voies-caduques)

Les voies actuelles ont une emprise géométrique contrairement aux anciennes voies.

* `vpt` pour "ville paris thoroughfares"
* `vpta` pour "ville paris thoroughfares actuelles"
* `vptc` pour "ville paris thoroughfares caduques"

In [ ]:
# Nom du répertoire où sont stockés et construits les triplets des factoïdes des données de la BAN
vpt_repository_name = "factoids_ville_de_paris"

# Fichier CSV pour stocker le résultat de la requête de la sélection
vpta_csv_file_name = "denominations-emprises-voies-actuelles.csv"
vpta_csv_file = os.path.join(data_folder, vpta_csv_file_name)
vptc_csv_file_name = "denominations-des-voies-caduques.csv"
vptc_csv_file = os.path.join(data_folder, vptc_csv_file_name)

# Propriétés liées au fichier `vpta_csv_file` (colonne décrivant la géométrie, séparateur de colonnes...)
vpta_csv_file_delimiter = ";"
vpta_csv_file_geom_col_name = "geo_shape"

# Le fichier `vpta_csv_file_name` sera modifié durant le traitement. Le fichier ci-dessous sera la version modifiée.
vpta_mod_csv_file_name = "denominations-emprises-voies-actuelles_wkt.csv"
vpta_mod_csv_file = os.path.join(tmp_folder, vpta_mod_csv_file_name)

# Fichier de mapping pour transformer le fichier CSV en fichier TTL
vpta_mapping_file_name = "mapping_voies_paris_actuelles.json"
vpta_mapping_file = os.path.join(mapping_folder, vpta_mapping_file_name)
vptc_mapping_file_name = "mapping_voies_paris_caduques.json"
vptc_mapping_file = os.path.join(mapping_folder, vptc_mapping_file_name)

# Fichier TTL pour structurer les connaissances des voies de Paris
vpta_kg_file_name = "voies_paris_actuelles.ttl"
vpta_kg_file = os.path.join(tmp_folder, vpta_kg_file_name)
vptc_kg_file_name = "voies_paris_caduques.ttl"
vptc_kg_file = os.path.join(tmp_folder, vptc_kg_file_name)

# Fichier TTL final des factoïdes de Ville de Paris
vpt_factoids_kg_file_name = "vpt_factoids.ttl"
vpt_factoids_kg_file = os.path.join(tmp_folder, vpt_factoids_kg_file_name)
vpt_permanent_kg_file_name = "vpt_permanent.ttl"
vpt_permanent_kg_file = os.path.join(tmp_folder, vpt_permanent_kg_file_name)

### Base Adresse Nationale (BAN)

Données de la [Base Adresse Nationale (BAN)](https://adresse.data.gouv.fr/base-adresse-nationale), accessible [ici](https://adresse.data.gouv.fr/data/ban/adresses/latest/csv)

`bpa` pour "BAN paris addresses"

In [ ]:
# Nom du répertoire où sont stockés et construits les triplets des factoïdes des données de la BAN
bpa_repository_name = "factoids_ban"

# Fichier CSV pour stocker le résultat de la requête de la sélection
bpa_csv_file_name = "ban_adresses.csv"
bpa_csv_file = os.path.join(data_folder, bpa_csv_file_name)

# Fichier de mapping pour transformer le fichier CSV en fichier TTL
bpa_mapping_file_name = "mapping_ban_adresses.json"
bpa_mapping_file = os.path.join(mapping_folder, bpa_mapping_file_name)

# Fichier TTL pour structurer les connaissances des voies de Paris
bpa_kg_file_name = "ban_adresses.ttl"
bpa_kg_file = os.path.join(tmp_folder, bpa_kg_file_name)

# Fichier TTL final des factoïdes de Ville de Paris
bpa_factoids_kg_file_name = "ban_factoids.ttl"
bpa_factoids_kg_file = os.path.join(tmp_folder, bpa_factoids_kg_file_name)
bpa_permanent_kg_file_name = "ban_permanent.ttl"
bpa_permanent_kg_file = os.path.join(tmp_folder, bpa_permanent_kg_file_name)

### OpenStreetMap (OSM)

Extraction des données d'OpenStreetMap

In [ ]:
# Nom du répertoire où sont stockés et construits les triplets des factoïdes des données d'OSM
osm_repository_name = "factoids_osm"

# Fichiers CSV pour stocker les résultats des requêtes de la sélection
osm_csv_file_name = "osm_adresses.csv"
osm_csv_file = os.path.join(data_folder, osm_csv_file_name)
osm_hn_csv_file_name = "osm_hn_adresses.csv"
osm_hn_csv_file = os.path.join(data_folder, osm_hn_csv_file_name)

# Fichiers de mapping pour transformer les fichier CSV en fichier TTL
osm_mapping_file_name = "mapping_osm_adresses.json"
osm_mapping_file = os.path.join(mapping_folder, osm_mapping_file_name)
osm_hn_mapping_file_name = "mapping_osm_hn_adresses.json"
osm_hn_mapping_file = os.path.join(mapping_folder, osm_hn_mapping_file_name)

# Fichiers TTL pour structurer les connaissances venant d'OSM
osm_kg_file_name = "osm_adresses.ttl"
osm_kg_file = os.path.join(tmp_folder, osm_kg_file_name)
osm_hn_kg_file_name = "osm_adresses_hn.ttl"
osm_hn_kg_file = os.path.join(tmp_folder, osm_hn_kg_file_name)

# Fichier TTL final des factoïdes de Ville de Paris
osm_factoids_kg_file_name = "osm_factoids.ttl"
osm_factoids_kg_file = os.path.join(tmp_folder, osm_factoids_kg_file_name)
osm_permanent_kg_file_name = "osm_permanent.ttl"
osm_permanent_kg_file = os.path.join(tmp_folder, osm_permanent_kg_file_name)

## Fonctions pour des traitements de construction des graphes de factoïdes

### Ajout de labels normalisés

* Étant donné les variations de labels qui sont similaires, la normalisation consiste ici à :
    * la mise en minuscule des caractères
    * suppression des signes diacritiques (accents, cédille...)
    * suppression de termes "inutiles" : déterminants, prépositions...
* Ces labels normalisés sont liés aux repères via `skos:hiddenLabel`

In [ ]:
def add_normalized_label_for_landmarks(graphdb_url, repository_name, factoids_graph_uri:URIRef):
    """
    Ajout de labels normalisés pour les repères via `skos:hiddenLabel` dans le graphe nommé temporaire `tmp_graph_uri`
    """

    label_var = "?label"
    norm_label_var = "?normLabel"
    norm_label_function = sp.get_lower_simplified_french_street_name_function(label_var)

    prefixes = """
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/>
    PREFIX atype: <http://rdf.geohistoricaldata.org/id/codes/address/attributeType/>
    """

    query = prefixes + f"""
    INSERT {{
        GRAPH ?g {{
            ?landmark skos:hiddenLabel {norm_label_var}.
        }}
    }}
    WHERE {{
        BIND({factoids_graph_uri.n3()} AS ?g)
        GRAPH ?g {{
            ?landmark a addr:Landmark; addr:hasAttribute [a addr:Attribute; addr:isAttributeType atype:Name; addr:hasAttributeVersion [a addr:AttributeVersion; addr:versionValue {label_var}]].
            BIND(REPLACE({norm_label_function}, " ", "") AS {norm_label_var})
        }}
    }}
    """

    gd.update_query(query, graphdb_url, repository_name)

### Suppression des instants temporels sans timeStamp

Il existe des resources de classe `addr:TimeInstant` qui ont un calendrier et une granularité dans avoir de time stamp. Il faut supprimer ces ressources qui n'ont aucune utilité.

In [ ]:
def remove_time_instant_without_timestamp(graphdb_url, repository_name, factoids_graph_uri:URIRef):
    query = f"""
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#> 
    DELETE {{
        ?timeInstant ?p ?o.
        ?s ?p ?timeInstant.
    }}
    WHERE {{
        ?timeInstant a addr:TimeInstant.
        MINUS {{?timeInstant addr:timeStamp ?timeStamp}}
        {{?timeInstant ?p ?o}}UNION{{?s ?p ?timeInstant}}
    }}
    """

    gd.update_query(query, graphdb_url, repository_name)

### Transfert de triplets vers le graphe nommé permanent

Tous les triplets créés via Ontotext-Refine sont initialement importés dans le graphe nommé des factoïdes. Certains doivent être mis dans le graphe nommé permanent, ce sont ceux qui ne pourront être modifiés à l'import dans le répertoire des faits. 

In [ ]:
def transfert_immutable_triples(graphdb_url, repository_name, factoids_graph_uri, permanent_graph_uri):
    prefixes = """
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>
    PREFIX prov: <http://www.w3.org/ns/prov#>
    PREFIX geofla: <http://data.ign.fr/def/geofla#>
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
    PREFIX ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/>
    PREFIX wb: <http://wikiba.se/ontology#>
    """ 
    
    # Déplacer tous les triplets dont le prédicat est `rico:isOrWasDescribedBy` dans le graphe nommé des triplets permenents
    query1 = prefixes + f"""
    DELETE {{
       ?s ?p ?o
    }}
    INSERT {{
        GRAPH {permanent_graph_uri.n3()} {{
            ?s ?p ?o.
        }}
    }}
    WHERE {{
        BIND(rico:isOrWasDescribedBy AS ?p)
        ?s ?p ?o.
    }} ; 
    """

    # Transférer dans le graphe permanent les triplets dont l'objet est une URI donnant la provenance 
    query2 = prefixes + f"""
    DELETE {{
        GRAPH ?gf {{ ?prov ?p ?o }}
    }}
    INSERT {{
        GRAPH ?gp {{ ?prov ?p ?o }}
    }}
    WHERE
    {{
        BIND({factoids_graph_uri.n3()} AS ?gf)
        BIND({permanent_graph_uri.n3()} AS ?gp)
        GRAPH ?gf {{ 
            ?elem prov:wasDerivedFrom ?prov.
            ?prov ?p ?o.
        }}
    }}
    """

    # Transférer dans le graphe permanent les triplets dont le sujet est un Item ou un Statement de Wikibase
    query3 = prefixes + f"""
    DELETE {{
        GRAPH ?gf {{ ?elem a ?type }}
    }}
    INSERT {{
        GRAPH ?gp {{ ?elem a ?type }}
    }}
    WHERE
    {{
        BIND({factoids_graph_uri.n3()} AS ?gf)
        BIND({permanent_graph_uri.n3()} AS ?gp)
        GRAPH ?gf {{ 
            ?elem a ?type.
        }}
        FILTER (?type in (wb:Item, wb:Statement))
    }}
    """

    queries = [query1, query2, query3]
    for query in queries:
        gd.update_query(query, graphdb_url, repository_name)

### Ajout d'un lien entre les repères et les sources

Un factoïde est la représentation d'une information, d'un fait dans une source. Ici, on crée des repères qui ont une identité définie par la source. Ces derniers doivent avoir un lien avec la source afin d'attester la provenance de son existence.

Pour cela, on sélectionne l'ensemble des repères (`?landmark`) situés dans le graphe nommé défini par `factoids_graph_uri` afin de créer le triplet `<?landmark rico:isOrWasDescribedBy ?sourceUri>` où `?sourceUri` est l'URI décrivant la source.

In [ ]:
def add_factoids_resources_links(graphdb_url, repository_name, factoids_graph_uri:URIRef):
    # query = f"""
    # PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#> 
    # PREFIX facts: <http://rdf.geohistoricaldata.org/id/address/facts/>
    # PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>

    # INSERT {{
    #     GRAPH ?g {{
    #         ?elem rico:isOrWasDescribedBy {factoids_refactoids_uri.n3()}.
    #     }}
    # }}WHERE {{
    #     {{?type rdfs:subClassOf* addr:Landmark}} UNION
    #     {{?type rdfs:subClassOf* addr:LandmarkRelation}} UNION
    #     {{?type rdfs:subClassOf* addr:AttributeVersion}} UNION
    #     {{?type rdfs:subClassOf* addr:Address}} UNION
    #     {{?type rdfs:subClassOf* addr:Change}} UNION
    #     {{?type rdfs:subClassOf* addr:Event}} UNION
    #     {{?type rdfs:subClassOf* addr:TemporalEntity}} UNION
    #     {{?type rdfs:subClassOf* rico:Record}} UNION
    #     {{?type rdfs:subClassOf* rico:CorporateBody}}
        
    #     BIND({factoids_graph_uri.n3()} AS ?g)
    #     GRAPH ?g {{
    #         ?elem a ?type.
    #     }}
    # }}
    # """

    # gd.update_query(query, graphdb_url, repository_name)

    prefixes = """
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>
    PREFIX prov: <http://www.w3.org/ns/prov#>
    PREFIX geofla: <http://data.ign.fr/def/geofla#>
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
    PREFIX ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/>
    PREFIX ban: <http://rdf.geohistoricaldata.org/id/address/sources/ban/>
    PREFIX wb: <http://wikiba.se/ontology#>
    """

    # Ajouter le lien de provenance des versions d'attributs
    query = prefixes + f"""
    INSERT {{
        GRAPH {factoids_graph_uri.n3()} {{
            ?attrVers ?p ?prov.
        }}
    }}
    WHERE {{
        BIND(prov:wasDerivedFrom AS ?p)
        ?lm a addr:Landmark; addr:hasAttribute [addr:hasAttributeVersion ?attrVers]; ?p ?prov.
    }} ; 
    """

    gd.update_query(query, graphdb_url, repository_name)

### Initialisation d'un répertoire pour créer un graphe de factoïdes

In [ ]:
def create_factoid_repository(graphdb_url, repository_name, namespace_prefixes, tmp_folder, ont_file, ontology_named_graph_name, ruleset_file=None, disable_same_as=False, clear_if_exists=False):
    "`clear_if_exists` is a bool to remove all statements if repository already exists"
    local_config_file_name = f"config_for_{repository_name}.ttl" 
    local_config_file = os.path.join(tmp_folder, local_config_file_name)

    if ruleset_file is not None:
        gd.create_config_local_repository_file(local_config_file, repository_name, ruleset_file=ruleset_file, disable_same_as=disable_same_as)
    else:
        gd.create_config_local_repository_file(local_config_file, repository_name, disable_same_as=disable_same_as)
    gd.create_repository_from_config_file(graphdb_url, local_config_file)
    if clear_if_exists:
        gd.clear_repository(graphdb_url, repository_name)

    gd.add_prefixes_to_repository(graphdb_url, repository_name, namespace_prefixes)
    gd.import_ttl_file_in_graphdb(graphdb_url, repository_name, ont_file, ontology_named_graph_name)

### Transfert des factoïdes vers le graphe des faits

In [ ]:
def transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, factoids_repository_name, factoids_ttl_file, permanent_ttl_file, factoids_graph_name, facts_graph_name, permanent_graph_name):
    factoids_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, factoids_repository_name, factoids_graph_name))
    permanent_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, factoids_repository_name, permanent_graph_name))
    gd.export_data_from_repository(graphdb_url, factoids_repository_name, factoids_ttl_file, factoids_graph_uri)
    gd.export_data_from_repository(graphdb_url, factoids_repository_name, permanent_ttl_file, permanent_graph_uri)
    gd.import_ttl_file_in_graphdb(graphdb_url, facts_repository_name, factoids_ttl_file, factoids_graph_name)
    gd.import_ttl_file_in_graphdb(graphdb_url, facts_repository_name, permanent_ttl_file, facts_graph_name)

### Conversion du fichier brut vers le graphe dans GraphDB

À partir un fichier brut (un fichier tabulaire comme un CSV), la fonction le convertit en graphe de connaissances dans un fichier `ttl` (ici `kg_file`). La manière de convertir le fichier est défini par le fichier `ontorefine_mapping_file`, la conversion se fait par Ontotext Refine. Par la suite, le fichier `ttl` est importé dans le répertoire dont le nom est `repository_name`, plus précisément dans le graphe nommé `graph_name`.

In [ ]:
def from_raw_to_data_to_graphdb(graphdb_url, ontorefine_url, ontorefine_cmd, repository_name, graph_name, csv_file, ontorefine_mapping_file, kg_file):
    # Si ça ne marche pas ici, c'est sûrement qu'Ontotext Refine n'est pas lancé
    otr.get_export_file_from_ontorefine(csv_file, ontorefine_mapping_file, kg_file, ontorefine_cmd, ontorefine_url, repository_name)

    # Importer le fichier `kg_file` qui a été créé lors de la ligne précédente dans le répertoire `repository_name`, dans le graphe nommé `graph_name` 
    gd.import_ttl_file_in_graphdb(graphdb_url, repository_name, kg_file, graph_name)

## Construction des factoïdes pour chaque source

### Nomenclature des voies de la ville de Paris

#### Création des données liées à la source

In [ ]:
def create_source_ville_paris(graphdb_url, repository_name, source_uri:URIRef, graph_uri:URIRef):
    query = f"""
        PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#> 
        PREFIX facts: <http://rdf.geohistoricaldata.org/id/address/facts/>
        PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>

        INSERT DATA {{
            GRAPH {graph_uri.n3()} {{
                {source_uri.n3()} a rico:Record;
                    rdfs:label "dénomination des voies de Paris (actuelles et caduques)"@fr;
                    rico:hasPublisher facts:DirTopoDocFoncVP .
                facts:DirTopoDocFoncVP a rico:CorporateBody;
                    rdfs:label "Département de la Topographie et de la Documentation Foncière de la Ville de Paris"@fr.    
            }}
        }}
        """
    
    gd.update_query(query, graphdb_url, repository_name)

#### Conversion du fichier de départ

Le fichier `vpta_csv_file` comporte une colonne donnant la géométrie de la voie (donnée ici par la variable `vpta_csv_file_geom_col_name`). Toutefois, les géométries sont représentées en geojson mais il est nécessaire de les avoir en WKT. La fonction ci-dessous permet de faire la conversion en enregistrant la nouvelle version sous un nouveau fichier.

In [ ]:
def get_csv_file_with_wkt_geom(csv_file_raw, csv_file, geom_column, delimiter=","):
    """
    Obtention d'une nouvelle version d'un fichier csv en convertissant une des colonnes (`geom_column`) en géométrie WKT si celle-ci est une géométrie geojson
    Le fichier de départ est `csv_file_raw` et la nouvelle version est `csv_file`.
    """
    file1 = open(csv_file_raw)
    file2 = open(csv_file, "w")
    csvreader1 = csv.reader(file1, delimiter=delimiter)
    csvreader2 = csv.writer(file2, delimiter=delimiter)

    header = next(csvreader1)
    csvreader2.writerow(header)
    geom_index = header.index(geom_column)

    for row in csvreader1:
        str_geom = row[geom_index]
        geom = json.loads(str_geom)
        wkt_geom = gp.from_geojson_to_wkt(geom)
        row[geom_index] = wkt_geom
        csvreader2.writerow(row)

    file1.close()
    file2.close()

#### Définition d'un processus de création des données de la Ville de Paris

In [ ]:
def create_factoid_process_ville_paris(graphdb_url, repository_name, namespace_prefixes, tmp_folder,
                                       ontorefine_url, ontorefine_cmd,
                                       ont_file, ontology_named_graph_name,
                                       factoids_graph_name, permanent_graph_name,
                                       vpta_csv_file, vpta_mod_csv_file,
                                       vpta_csv_file_delimiter, vpta_csv_file_geom_col_name, vptc_csv_file,
                                       vpta_mapping_file, vptc_mapping_file,
                                       vpta_kg_file, vptc_kg_file):
    """
    Fonction pour faire l'ensemble des processus relatifs à la création des factoïdes pour les données de la dénomination des voies de la ville de Paris
    """

    # Création d'un répertoire des factoïdes pour les données de la ville de Paris
    create_factoid_repository(graphdb_url, repository_name, namespace_prefixes, tmp_folder, ont_file, ontology_named_graph_name, disable_same_as=False, clear_if_exists=True)

    # Récupération des URI des graphes nommés
    factoids_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, repository_name, factoids_graph_name))
    permanent_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, repository_name, permanent_graph_name))

    # Création d'une nouvelle version du fichier `vpta_csv_file` via `vpta_mod_csv_file` qui comporte des géométries WKT au lieu de geojson
    get_csv_file_with_wkt_geom(vpta_csv_file, vpta_mod_csv_file, vpta_csv_file_geom_col_name, vpta_csv_file_delimiter)

    # A partir des fichiers csv décrivant les voies de la ville de Paris, convertir en un graphe de connaissance selon le mapping défini
    # Puis import du graphe dans le répertoire dont le nom est `repository_name` et dans le graphe nommé donné par `graph_name`
    from_raw_to_data_to_graphdb(graphdb_url, ontorefine_url, ontorefine_cmd, repository_name, factoids_graph_name, vpta_mod_csv_file, vpta_mapping_file, vpta_kg_file)
    from_raw_to_data_to_graphdb(graphdb_url, ontorefine_url, ontorefine_cmd, repository_name, factoids_graph_name, vptc_csv_file, vptc_mapping_file, vptc_kg_file)

    # Suppression des instants qui n'ont aucun timeStamp (instants sans date)
    remove_time_instant_without_timestamp(graphdb_url, repository_name, factoids_graph_uri)
    
    # L'URI ci-dessous définit la source liée à la ville de Paris
    vdp_source_uri = URIRef("http://rdf.geohistoricaldata.org/id/address/facts/Source_VDP")
    create_source_ville_paris(graphdb_url, repository_name, vdp_source_uri, permanent_graph_uri)

    # # Ajout de labels normalisés
    add_normalized_label_for_landmarks(graphdb_url, repository_name, factoids_graph_uri)

    # Transfert de triplets non modifiables vers le graphe nommé permanent
    transfert_immutable_triples(graphdb_url, repository_name, factoids_graph_uri, permanent_graph_uri)

    # # Ajout de liens entre les ressources de type repère et la source
    # add_factoids_resources_links(graphdb_url, repository_name, factoids_graph_uri)

### Base Adresse Nationale (BAN)

#### Création des données liées à la source

In [ ]:
def create_source_ban(graphdb_url, repository_name, source_uri:URIRef, graph_uri:URIRef):
    query = f"""
        PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#> 
        PREFIX facts: <http://rdf.geohistoricaldata.org/id/address/facts/>
        PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>

        INSERT DATA {{
            GRAPH {graph_uri.n3()} {{
                {source_uri.n3()} a rico:Record;
                    rdfs:label "Base Adresse Nationale"@fr;
                    rico:hasPublisher facts:DINUM_ANCT_IGN .
                facts:DINUM_ANCT_IGN a rico:CorporateBody;
                    rdfs:label "DINUM / ANCT / IGN"@fr.
            }}
        }}
        """
    
    gd.update_query(query, graphdb_url, repository_name)

#### Nettoyage du graphe

Détection de repères décrits via plusieurs ressources et création d'une unique ressource pour chaque.

In [ ]:
# Détection des communes et arrondissements dupliqués et création d'une source centrale (?newLandmark) selon le code INSEE.
def clean_ban_graph(graphdb_url, repository_name, factoids_graph_uri, permanent_graph_uri):
    prefixes = """
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX prov: <http://www.w3.org/ns/prov#>
    PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>
    PREFIX geofla: <http://data.ign.fr/def/geofla#>
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
    PREFIX ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/>
    PREFIX ban: <http://rdf.geohistoricaldata.org/id/address/sources/ban/>
    """

    label_var = "?label"
    norm_label_var = "?normLabel"
    norm_label_function = sp.get_lower_simplified_french_street_name_function(label_var)

    query1 = prefixes + f"""
    INSERT {{
        GRAPH ?g {{
            ?landmark skos:hiddenLabel {norm_label_var}.
        }}
    }}
    WHERE {{
        BIND({factoids_graph_uri.n3()} AS ?g)
        GRAPH ?g {{
            ?landmark a addr:Landmark; rdfs:label {label_var}.
            BIND(REPLACE({norm_label_function}, " ", "") AS {norm_label_var})
        }}
    }}
    """

    query2 = prefixes + f"""
    INSERT {{
        GRAPH ?g {{
            ?landmark skos:exactMatch ?tmpLandmark.
        }}
    }}
    WHERE
    {{
        BIND({factoids_graph_uri.n3()} AS ?g)
        {{
            SELECT DISTINCT ?insee {{
                GRAPH ?g {{
                    ?tmpLandmark a addr:Landmark; geofla:numInsee ?insee.
                }}
            }}
        }}
        BIND(URI(CONCAT(STR(URI(ban:)), "BAN_LM_", STRUUID())) AS ?landmark)

        GRAPH ?g {{
            ?tmpLandmark a addr:Landmark; addr:isLandmarkType ?landmarkType; geofla:numInsee ?insee.
        }}
    }}
    """

    # Détection des doublons sur les codes postaux

    query3 = prefixes + f"""
    INSERT {{
        GRAPH ?g {{
            ?landmark skos:exactMatch ?tmpLandmark.
        }}
    }}
    WHERE
    {{
        BIND({factoids_graph_uri.n3()} AS ?g)
        {{
            SELECT DISTINCT ?postalCode {{
                GRAPH ?g {{
                    ?tmpLandmark a addr:Landmark; addr:isLandmarkType ltype:PostalCode; rdfs:label ?postalCode.
                }}
            }}
        }}
        BIND(URI(CONCAT(STR(URI(ban:)), "BAN_LM_", STRUUID())) AS ?landmark)

        GRAPH ?g {{
            ?tmpLandmark a addr:Landmark; addr:isLandmarkType ltype:PostalCode; rdfs:label ?postalCode.
        }}
    }}
    """

    # Détection des doublons sur les voies (même nom et appartient à la même commune ou au même arrondissement)

    query4 = prefixes + f"""
    INSERT {{
        GRAPH ?g {{
            ?landmark skos:exactMatch ?tmpLandmark.
        }}
    }}
    WHERE
    {{
        BIND({factoids_graph_uri.n3()} AS ?g)
        {{
            SELECT DISTINCT ?label ?district WHERE {{
                GRAPH ?g {{
                    ?tmpLandmark a addr:Landmark; addr:isLandmarkType ltype:Thoroughfare; skos:hiddenLabel ?label.
                    ?addrSeg a addr:AddressSegment; addr:relatum ?tmpLandmark; addr:nextStep [a addr:AddressSegment; addr:relatum ?tmpDistrict].
                    ?tmpDistrict a addr:Landmark; addr:isLandmarkType ltype:District.
                    ?district skos:exactMatch ?tmpDistrict.
                }}
            }}
        }}
        BIND(URI(CONCAT(STR(URI(ban:)), "BAN_LM_", STRUUID())) AS ?landmark)

        GRAPH ?g {{
            ?tmpLandmark a addr:Landmark; addr:isLandmarkType ltype:Thoroughfare; skos:hiddenLabel ?label.
                    ?addrSeg a addr:AddressSegment; addr:relatum ?tmpLandmark; addr:nextStep [a addr:AddressSegment; addr:relatum ?tmpDistrict].
                    ?tmpDistrict a addr:Landmark; addr:isLandmarkType ltype:District.
                    ?district skos:exactMatch ?tmpDistrict.
        }}
    }}
    """
    # Transfert des données des ressources temporaires vers les permanentes.
    query5 = prefixes + f"""
    DELETE {{
        GRAPH ?g {{
            ?s ?p ?tmpLandmark.
            ?tmpLandmark ?p ?o.
        }}
    }}
    INSERT {{
        GRAPH ?g {{
            ?s ?p ?landmark.
            ?landmark ?p ?o.
        }}
    }}
    WHERE {{
        ?landmark skos:exactMatch ?tmpLandmark.
        GRAPH ?g {{
            {{?tmpLandmark ?p ?o}} UNION {{?s ?p ?tmpLandmark}}
          }}
    }} ; 

    DELETE {{
        ?landmark skos:exactMatch ?tmpLandmark.
    }}
    WHERE {{
        BIND({factoids_graph_uri.n3()} AS ?g)
        GRAPH ?g {{
            ?landmark skos:exactMatch ?tmpLandmark.
        }}
    }}
    """

    queries = [query1, query2, query3, query4, query5]
    for query in queries:
        gd.update_query(query, graphdb_url, repository_name)

### Ajout d'événéments / de changements ainsi que des attributs à tous les repères

In [ ]:
def add_missing_elements_for_landmarks(graphdb_url, repository_name, factoids_graph_uri):
    """
    Ajouter des éléments comme les changements, les événements, les attributs et leurs versions
    """

    query = f"""
    PREFIX geo: <http://www.opengis.net/ont/geosparql#>
    PREFIX geofla: <http://data.ign.fr/def/geofla#>
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#> 
    PREFIX factoids: <http://rdf.geohistoricaldata.org/id/address/factoids/>
    PREFIX ctype: <http://rdf.geohistoricaldata.org/id/codes/address/changeType/>
    PREFIX atype: <http://rdf.geohistoricaldata.org/id/codes/address/attributeType/>
    DELETE {{
        GRAPH ?g {{ 
            ?landmark geo:asWKT ?geom; geofla:numInsee ?inseeCode.
        }}
    }}
    INSERT {{
        GRAPH ?g {{
            ?landmark addr:hasAttribute ?nameAttribute, ?geomAttribute, ?inseeCodeAttribute.
            ?nameAttribute a addr:Attribute; addr:isAttributeType atype:Name; addr:hasAttributeVersion ?versionNameAttribute.
            ?inseeCodeAttribute a addr:Attribute; addr:isAttributeType atype:InseeCode; addr:hasAttributeVersion ?versionInseeCodeAttribute.
            ?geomAttribute a addr:Attribute; addr:isAttributeType atype:Geometry; addr:hasAttributeVersion ?versionGeomAttribute.
            ?versionNameAttribute a addr:AttributeVersion; addr:versionValue ?label.
            ?versionInseeCodeAttribute a addr:AttributeVersion; addr:versionValue ?inseeCode.
            ?versionGeomAttribute a addr:AttributeVersion; addr:versionValue ?geom.
            ?landmarkChangeApp a addr:LandmarkChange; addr:isChangeType ctype:LandmarkAppearance; addr:appliedTo ?landmark; addr:dependsOn ?landmarkEventApp.
            ?landmarkChangeDis a addr:LandmarkChange; addr:isChangeType ctype:LandmarkDisappearance; addr:appliedTo ?landmark; addr:dependsOn ?landmarkEventDis.
            ?versNameAttributeChangeApp a addr:AttributeChange; addr:isChangeType ctype:AttributeVersionAppearance; addr:appliedTo ?nameAttribute; addr:dependsOn ?landmarkEventApp; addr:makesEffective ?versionNameAttribute.
            ?versNameAttributeChangeDis a addr:AttributeChange; addr:isChangeType ctype:AttributeVersionDisappearance; addr:appliedTo ?nameAttribute; addr:dependsOn ?landmarkEventDis; addr:outdates ?versionNameAttribute.
            ?inseeCodeAttributeChangeApp a addr:AttributeChange; addr:isChangeType ctype:AttributeVersionAppearance; addr:appliedTo ?inseeCodeAttribute; addr:dependsOn ?inseeCodeAttributeEventApp; addr:makesEffective ?versionInseeCodeAttribute.
            ?inseeCodeAttributeChangeDis a addr:AttributeChange; addr:isChangeType ctype:AttributeVersionDisappearance; addr:appliedTo ?inseeCodeAttribute; addr:dependsOn ?inseeCodeAttributeEventDis; addr:outdates ?versionInseeCodeAttribute.
            ?geomAttributeChangeApp a addr:AttributeChange; addr:isChangeType ctype:AttributeVersionAppearance; addr:appliedTo ?geomAttribute; addr:dependsOn ?geomAttributeEventApp; addr:makesEffective ?versionGeomAttribute.
            ?geomAttributeChangeDis a addr:AttributeChange; addr:isChangeType ctype:AttributeVersionDisappearance; addr:appliedTo ?geomAttribute; addr:dependsOn ?geomAttributeEventDis; addr:outdates ?versionGeomAttribute.
            ?landmarkEventApp a addr:Event.
            ?landmarkEventDis a addr:Event.
            ?inseeCodeAttributeEventApp a addr:Event.
            ?inseeCodeAttributeEventDis a addr:Event.
            ?geomAttributeEventApp a addr:Event.
            ?geomAttributeEventDis a addr:Event.
        }}  
    }}
    WHERE {{
        {{
            SELECT * {{
                BIND({factoids_graph_uri.n3()} AS ?g)
                GRAPH ?g {{
                    ?landmark a addr:Landmark; rdfs:label ?label.
                    OPTIONAL {{?landmark geo:asWKT ?geom}}
                    OPTIONAL {{?landmark geofla:numInsee ?inseeCode}}
                }}
            }}
        }}
        BIND(URI(CONCAT(STR(URI(factoids:)), "CGA_", STRUUID())) AS ?landmarkChangeApp)
        BIND(URI(CONCAT(STR(URI(factoids:)), "CGD_", STRUUID())) AS ?landmarkChangeDis)
        BIND(URI(CONCAT(STR(URI(factoids:)), "EVA_", STRUUID())) AS ?landmarkEventApp)
        BIND(URI(CONCAT(STR(URI(factoids:)), "EVD_", STRUUID())) AS ?landmarkEventDis)
        BIND(URI(CONCAT(STR(URI(factoids:)), "AN_", STRUUID())) AS ?nameAttribute)
        BIND(URI(CONCAT(STR(URI(factoids:)), "ANV_", STRUUID())) AS ?versionNameAttribute)
        BIND(URI(CONCAT(STR(URI(factoids:)), "CGA_AN_", STRUUID())) AS ?versNameAttributeChangeApp)
        BIND(URI(CONCAT(STR(URI(factoids:)), "CGD_AN_", STRUUID())) AS ?versNameAttributeChangeDis)
        BIND(IF(BOUND(?inseeCode), URI(CONCAT(STR(URI(factoids:)), "AI_", STRUUID())), ?x) AS ?inseeCodeAttribute)
        BIND(IF(BOUND(?inseeCode), URI(CONCAT(STR(URI(factoids:)), "AIV_", STRUUID())), ?x) AS ?versionInseeCodeAttribute)
        BIND(IF(BOUND(?inseeCode), URI(CONCAT(STR(URI(factoids:)), "CGA_AI_", STRUUID())), ?x) AS ?inseeCodeAttributeChangeApp)
        BIND(IF(BOUND(?inseeCode), URI(CONCAT(STR(URI(factoids:)), "CGD_AI_", STRUUID())), ?x) AS ?inseeCodeAttributeChangeDis)
        BIND(IF(BOUND(?inseeCode), URI(CONCAT(STR(URI(factoids:)), "EVA_AI_", STRUUID())), ?x) AS ?inseeCodeAttributeEventApp)
        BIND(IF(BOUND(?inseeCode), URI(CONCAT(STR(URI(factoids:)), "EVD_AI_", STRUUID())), ?x) AS ?inseeCodeAttributeEventDis)
        BIND(IF(BOUND(?geom), URI(CONCAT(STR(URI(factoids:)), "AG_", STRUUID())), ?x) AS ?geomAttribute)
        BIND(IF(BOUND(?geom), URI(CONCAT(STR(URI(factoids:)), "AGV_", STRUUID())), ?x) AS ?versionGeomAttribute)
        BIND(IF(BOUND(?geom), URI(CONCAT(STR(URI(factoids:)), "CGA_AG_", STRUUID())), ?x) AS ?geomAttributeChangeApp)
        BIND(IF(BOUND(?geom), URI(CONCAT(STR(URI(factoids:)), "CGD_AG_", STRUUID())), ?x) AS ?geomAttributeChangeDis)
        BIND(IF(BOUND(?geom), URI(CONCAT(STR(URI(factoids:)), "EVA_AG_", STRUUID())), ?x) AS ?geomAttributeEventApp)
        BIND(IF(BOUND(?geom), URI(CONCAT(STR(URI(factoids:)), "EVD_AG_", STRUUID())), ?x) AS ?geomAttributeEventDis)
    }}
    """
    
    gd.update_query(query, graphdb_url, repository_name)


#### Définition d'un processus de création des données de la BAN

In [ ]:
def create_factoid_process_ban(graphdb_url, repository_name, namespace_prefixes, tmp_folder,
                               ontorefine_url, ontorefine_cmd, ont_file, ontology_named_graph_name,
                               factoids_graph_name, permanent_graph_name,
                               ban_csv_file, ban_mapping_file, ban_kg_file):

    """
    Fonction pour faire l'ensemble des processus relatifs à la création des factoïdes pour les données de la BAN
    """

    # Création d'un répertoire des factoïdes pour les données de la BAN
    create_factoid_repository(graphdb_url, repository_name, namespace_prefixes, tmp_folder, ont_file, ontology_named_graph_name, disable_same_as=False, clear_if_exists=True)

    # Récupération des URI des graphes nommés
    factoids_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, repository_name, factoids_graph_name))
    permanent_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, repository_name, permanent_graph_name))

    # A partir des fichiers csv décrivant les adresses de la BAN dans Paris, convertir en un graphe de connaissance selon le mapping défini
    # Puis import du graphe dans le répertoire dont le nom est `repository_name` et dans le graphe nommé donné par `graph_name`
    from_raw_to_data_to_graphdb(graphdb_url, ontorefine_url, ontorefine_cmd, repository_name, factoids_graph_name, ban_csv_file, ban_mapping_file, ban_kg_file)

    # Suppression des instants qui n'ont aucun timeStamp (instants sans date)
    remove_time_instant_without_timestamp(graphdb_url, repository_name, factoids_graph_uri)

    # Nettoyer les données en fusionnant les doublons après l'import dans GraphDB
    clean_ban_graph(graphdb_url, repository_name, factoids_graph_uri, permanent_graph_uri)

    # Ajout d'éléments manquants
    add_missing_elements_for_landmarks(graphdb_url, repository_name, factoids_graph_uri)

    # L'URI ci-dessous définit la source liée à la BAN
    ban_factoids_uri = URIRef("http://rdf.geohistoricaldata.org/id/address/facts/Source_BAN")
    create_source_ban(graphdb_url, repository_name, ban_factoids_uri, permanent_graph_uri)

    # Transfert de triplets non modifiables vers le graphe nommé permanent
    transfert_immutable_triples(graphdb_url, repository_name, factoids_graph_uri, permanent_graph_uri)

    # Ajout de labels normalisés
    add_normalized_label_for_landmarks(graphdb_url, repository_name, factoids_graph_uri)
    
    # Ajout de liens entre les ressources de type repère et la source
    add_factoids_resources_links(graphdb_url, repository_name, factoids_graph_uri)

### Voies de Paris via Wikidata

#### Création des données liées à la source

In [ ]:
def create_source_wikidata(graphdb_url, repository_name, source_uri:URIRef, graph_uri:URIRef):
    query = f"""
        PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#> 
        PREFIX facts: <http://rdf.geohistoricaldata.org/id/address/facts/>
        PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>

        INSERT DATA {{
            GRAPH {graph_uri.n3()} {{
                {source_uri.n3()} a rico:Record;
                    rdfs:label "Wikidata"@fr.
            }}
        }}
        """
    
    gd.update_query(query, graphdb_url, repository_name)

#### Nettoyage du graphe

In [ ]:
def clean_wikidata_graph(graphdb_url, repository_name, factoids_graph_uri:URIRef, permanent_graph_uri:URIRef):
    prefixes = """
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX geofla: <http://data.ign.fr/def/geofla#>
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
    PREFIX time: <http://www.w3.org/2006/time#> 
    PREFIX ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/>
    PREFIX ban: <http://rdf.geohistoricaldata.org/id/address/sources/ban/>
    PREFIX wb: <http://wikiba.se/ontology#>
    """

    query1 =  prefixes + f"""
    INSERT DATA{{
        GRAPH {permanent_graph_uri.n3()} {{
            wb:Statement a owl:Class.
            wb:Item a owl:Class.
        }}
    }}
    """

    queries = [query1]
    for query in queries:
        gd.update_query(query, graphdb_url, repository_name)

#### Sélection des voies de Paris sur Wikidata via une requête de sélection

Via une requête SPARQL de sélection, on récupère une liste de ressources à créér :
* pour chaque ressource décrivant une voie dans Paris, on créé autant de ressources qu'elle a de nom officiel. La place de la Nation ([wd:Q1573359](https://www.wikidata.org/entity/Q1573359)) a quatre noms officiels donc on aura 4 ressources. Pour celles qui n'ont pas de nom officiel, on créé une seule ressource dont le nom est le label en français
* ces ressources créées seront liées à celle de Wikidata dont elle provient via `skos:closeMatch`
* chaque ressource créée suit la structure définie par l'ontologie
* le résultat de la requête est stocké dans un fichier csv qui sera converti en graphe de connaissance via Ontotext Refine et un fichier de mapping 

In [ ]:
def get_paris_thoroughfares_from_wikidata(out_csv_file):
   query = """
   PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#> 
   PREFIX source: <http://rdf.geohistoricaldata.org/id/address/sources/wikidata/> 
   PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
   PREFIX owl: <http://www.w3.org/2002/07/owl#>
   PREFIX wd: <http://www.wikidata.org/entity/>
   PREFIX wdt: <http://www.wikidata.org/prop/direct/>
   PREFIX pq: <http://www.wikidata.org/prop/qualifier/>
   PREFIX ps: <http://www.wikidata.org/prop/statement/>
   PREFIX p: <http://www.wikidata.org/prop/>
   PREFIX pqv: <http://www.wikidata.org/prop/qualifier/value/>
   PREFIX wb: <http://wikiba.se/ontology#>
   PREFIX time: <http://www.w3.org/2006/time#>
   SELECT ?landmarkQid ?landmarkId ?landmarkType ?attributeNameVersionId ?nomOff ?dateStartStamp ?dateStartPrec ?dateStartCal ?dateEndStamp ?dateEndPrec ?dateEndCal ?statement ?statementType
   WHERE {
   BIND(CONCAT("LM_", STRUUID()) AS ?landmarkId)
   BIND(CONCAT("AN_", STRUUID()) AS ?attributeNameVersionId)
   {
      SELECT DISTINCT * WHERE {
         {?landmarkQid p:P361 [ps:P361 wd:Q16024163].}UNION{?landmarkQid p:P361 [ps:P361 wd:Q107311481].}
         {?landmarkQid p:P1448 ?nomOffSt.
         ?nomOffSt ps:P1448 ?nomOff.
         BIND(?nomOffSt AS ?statement)
         BIND(wb:Statement AS ?statementType)
         OPTIONAL {?nomOffSt pq:P580 ?dateStartStamp; pqv:P580 [wb:timePrecision ?dateStartPrecRaw; wb:timeCalendarModel ?dateStartCal]}
         OPTIONAL {?nomOffSt pq:P582 ?dateEndStamp; pqv:P582 [wb:timePrecision ?dateEndPrecRaw; wb:timeCalendarModel ?dateEndCal]}
         }UNION{
         ?landmarkQid rdfs:label ?nomOff.
         FILTER (LANG(?nomOff) = "fr")
         MINUS {?landmarkQid p:P1448 ?nomOffSt}
         BIND(?landmarkQid AS ?statement)
         BIND(wb:Item AS ?statementType)
         }
         BIND("Thoroughfare" AS ?landmarkType)
      }
   }

   BIND(IF(?dateStartPrecRaw = 11, time:unitDay, 
            IF(?dateStartPrecRaw = 10, time:unitMonth,
               IF(?dateStartPrecRaw = 9, time:unitYear,
                  IF(?dateStartPrecRaw = 8, time:unitDecade,
                     IF(?dateStartPrecRaw = 7, time:unitCentury,
                        IF(?dateStartPrecRaw = 6, time:unitMillenium, ?x
                           )))))) AS ?dateStartPrec)
   BIND(IF(?dateEndPrecRaw = 11, time:unitDay, 
            IF(?dateEndPrecRaw = 10, time:unitMonth,
               IF(?dateEndPrecRaw = 9, time:unitYear,
                  IF(?dateEndPrecRaw = 8, time:unitDecade,
                     IF(?dateEndPrecRaw = 7, time:unitCentury,
                        IF(?dateEndPrecRaw = 6, time:unitMillenium, ?x
                           )))))) AS ?dateEndPrec)
   }
    """

   wd.save_select_query_as_csv_file(query, out_csv_file)

In [ ]:
def get_areas_around_paris_from_wikidata(out_csv_file):
    """
    Sélection de zones autour de Paris (communes de l'ancien département de la Seine) et des quartiers / arrondissements de la capitale française"""
    
    query = """
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#> 
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    PREFIX pq: <http://www.wikidata.org/prop/qualifier/>
    PREFIX ps: <http://www.wikidata.org/prop/statement/>
    PREFIX p: <http://www.wikidata.org/prop/>
    PREFIX pqv: <http://www.wikidata.org/prop/qualifier/value/>
    PREFIX wb: <http://wikiba.se/ontology#>
    PREFIX time: <http://www.w3.org/2006/time#>

    SELECT ?landmarkQid ?landmarkId ?landmarkType ?attributeInseeCodeVersionId ?inseeCodeSt ?inseeCode ?inseeDateStartStamp ?inseeDateStartPrec ?inseeDateStartCal ?inseeDateEndStamp ?inseeDateEndPrec ?inseeDateEndCal ?attributeNameVersionId ?nomOff ?dateStartStamp ?dateStartPrec ?dateStartCal ?dateEndStamp ?dateEndPrec ?dateEndCal ?statement ?statementType 
    {
        {
        SELECT DISTINCT * WHERE {
        BIND(CONCAT("LM_", STRUUID()) AS ?landmarkId)
        BIND(CONCAT("ANV_", STRUUID()) AS ?attributeNameVersionId)
        {
            SELECT DISTINCT * WHERE {
            {?landmarkQid p:P31 [ps:P31 wd:Q252916].}UNION{?landmarkQid p:P31 [ps:P31 wd:Q702842]; p:P131 [ps:P131 wd:Q90].}UNION{?landmarkQid p:P31 [ps:P31 wd:Q484170]; p:P131 [ps:P131 wd:Q1142326].}
            {?landmarkQid p:P1448 ?nomOffSt.
            ?nomOffSt ps:P1448 ?nomOff.
            BIND(?nomOffSt AS ?statement)
            BIND(wb:Statement AS ?statementType)
            OPTIONAL {?nomOffSt pq:P580 ?dateStartStamp; pqv:P580 [wb:timePrecision ?dateStartPrecRaw; wb:timeCalendarModel ?dateStartCal]}
            OPTIONAL {?nomOffSt pq:P582 ?dateEndStamp; pqv:P582 [wb:timePrecision ?dateEndPrecRaw; wb:timeCalendarModel ?dateEndCal]}
            }UNION{
                ?landmarkQid rdfs:label ?nomOff.
                FILTER (LANG(?nomOff) = "fr")
                MINUS {?landmarkQid p:P1448 ?nomOffSt}
                BIND(?landmarkQid AS ?statement)
                BIND(wb:Item AS ?statementType)
            }
            ?landmarkQid p:P31 [ps:P31 ?wdLandmarkType].
            BIND(
                IF(?wdLandmarkType = wd:Q252916, "District",
                IF(?wdLandmarkType = wd:Q702842, "District",
                    IF(?wdLandmarkType = wd:Q484170, "City", ?x))) AS ?landmarkType)
            FILTER(BOUND(?landmarkType))
            }
        }
        OPTIONAL {?landmarkQid p:P374 ?inseeCodeSt.
                    ?inseeCodeSt ps:P374 ?inseeCode.
                    OPTIONAL {?inseeCodeSt pq:P580 ?inseeDateStartStamp; pqv:P580 [wb:timePrecision ?inseeDateStartPrecRaw; wb:timeCalendarModel ?inseeDateStartCal]}
                    OPTIONAL {?inseeCodeSt pq:P582 ?inseeDateEndStamp; pqv:P582 [wb:timePrecision ?inseeDateEndPrecRaw; wb:timeCalendarModel ?inseeDateEndCal]}
                }
        }
    }
    BIND(IF(BOUND(?inseeCodeSt), CONCAT("AIV_", STRUUID()), ?x) AS ?attributeInseeCodeVersionId)

    BIND(IF(?dateStartPrecRaw = 11, time:unitDay, 
            IF(?dateStartPrecRaw = 10, time:unitMonth,
                IF(?dateStartPrecRaw = 9, time:unitYear,
                    IF(?dateStartPrecRaw = 8, time:unitDecade,
                    IF(?dateStartPrecRaw = 7, time:unitCentury,
                        IF(?dateStartPrecRaw = 6, time:unitMillenium, ?x
                            )))))) AS ?dateStartPrec)
    BIND(IF(?dateEndPrecRaw = 11, time:unitDay, 
            IF(?dateEndPrecRaw = 10, time:unitMonth,
                IF(?dateEndPrecRaw = 9, time:unitYear,
                    IF(?dateEndPrecRaw = 8, time:unitDecade,
                    IF(?dateEndPrecRaw = 7, time:unitCentury,
                        IF(?dateEndPrecRaw = 6, time:unitMillenium, ?x
                            )))))) AS ?dateEndPrec)
    BIND(IF(?inseeDateStartPrecRaw = 11, time:unitDay, 
            IF(?inseeDateStartPrecRaw = 10, time:unitMonth,
                IF(?inseeDateStartPrecRaw = 9, time:unitYear,
                    IF(?inseeDateStartPrecRaw = 8, time:unitDecade,
                    IF(?inseeDateStartPrecRaw = 7, time:unitCentury,
                        IF(?inseeDateStartPrecRaw = 6, time:unitMillenium, ?x
                            )))))) AS ?inseeDateStartPrec)
    BIND(IF(?inseeDateEndPrecRaw = 11, time:unitDay, 
            IF(?inseeDateEndPrecRaw = 10, time:unitMonth,
                IF(?inseeDateEndPrecRaw = 9, time:unitYear,
                    IF(?inseeDateEndPrecRaw = 8, time:unitDecade,
                    IF(?inseeDateEndPrecRaw = 7, time:unitCentury,
                        IF(?inseeDateEndPrecRaw = 6, time:unitMillenium, ?x
                            )))))) AS ?inseeDateEndPrec)
    }
    """

    query = wd.save_select_query_as_csv_file(query, out_csv_file)

In [ ]:
def get_paris_locations_from_wikidata(out_csv_file):
    """
    Obtenir la localisation des données (voies et zones) de Paris depuis Wikidata"""

    query = """
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    PREFIX pq: <http://www.wikidata.org/prop/qualifier/>
    PREFIX ps: <http://www.wikidata.org/prop/statement/>
    PREFIX p: <http://www.wikidata.org/prop/>
    PREFIX pqv: <http://www.wikidata.org/prop/qualifier/value/>
    PREFIX wb: <http://wikiba.se/ontology#>
    PREFIX time: <http://www.w3.org/2006/time#>

    SELECT DISTINCT ?landmarkRelationId ?locatumQid ?relatumQid ?landmarkRelationType ?dateStartStamp ?dateStartPrec ?dateStartCal ?dateEndStamp ?dateEndPrec ?dateEndCal ?relatumQidSt WHERE {
    {?locatumQid p:P361 [ps:P361 wd:Q16024163].}UNION{?locatumQid p:P361 [ps:P361 wd:Q107311481].}UNION{?locatumQid p:P31 [ps:P31 wd:Q252916].}UNION{?locatumQid p:P31 [ps:P31 wd:Q702842]; p:P131 [ps:P131 wd:Q90].}UNION{?locatumQid p:P31 [ps:P31 wd:Q484170]; p:P131 [ps:P131 wd:Q1142326].}
    ?locatumQid p:P131 ?relatumQidSt.
    ?relatumQidSt ps:P131 ?relatumQid. 
    OPTIONAL {?relatumQidSt pq:P580 ?dateStartStamp; pqv:P580 [wb:timePrecision ?dateStartPrecRaw; wb:timeCalendarModel ?dateStartCal]}
    OPTIONAL {?relatumQidSt pq:P582 ?dateEndStamp; pqv:P582 [wb:timePrecision ?dateEndPrecRaw; wb:timeCalendarModel ?dateEndCal]}
    BIND("Within" AS ?landmarkRelationType)
    BIND(CONCAT("LR_", STRUUID()) AS ?landmarkRelationId)
    BIND(IF(?dateStartPrecRaw = 11, time:unitDay, 
            IF(?dateStartPrecRaw = 10, time:unitMonth,
                IF(?dateStartPrecRaw = 9, time:unitYear,
                    IF(?dateStartPrecRaw = 8, time:unitDecade,
                    IF(?dateStartPrecRaw = 7, time:unitCentury,
                        IF(?dateStartPrecRaw = 6, time:unitMillenium, ?x
                            )))))) AS ?dateStartPrec)
    BIND(IF(?dateEndPrecRaw = 11, time:unitDay, 
            IF(?dateEndPrecRaw = 10, time:unitMonth,
                IF(?dateEndPrecRaw = 9, time:unitYear,
                    IF(?dateEndPrecRaw = 8, time:unitDecade,
                    IF(?dateEndPrecRaw = 7, time:unitCentury,
                        IF(?dateEndPrecRaw = 6, time:unitMillenium, ?x
                            )))))) AS ?dateEndPrec)
    }
    """
    
    query = wd.save_select_query_as_csv_file(query, out_csv_file)

In [ ]:
def transfert_landmark_relations_to_thoroughfares_wikidata(graphdb_url, repository_name, factoids_graph_uri:URIRef, tmp_graph_uri:URIRef):
    query = f"""
    PREFIX ctype: <http://rdf.geohistoricaldata.org/id/codes/address/changeType/>
    PREFIX lrtype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkRelationType/>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX prov: <http://www.w3.org/ns/prov#>
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
    PREFIX wikidata: <http://rdf.geohistoricaldata.org/id/address/sources/wikidata/>
    
    INSERT {{
        GRAPH ?gf {{
            ?landmarkRelation a addr:LandmarkRelation; addr:isLandmarkRelationType ?landmarkRelationType;
            addr:locatum ?cml; addr:relatum ?cmr; prov:wasDerivedFrom ?prov.
            ?lrChangeApp a addr:Change; addr:isChangeType ctype:LandmarkRelationAppearance; addr:appliedTo ?landmarkRelation; addr:dependsOn ?lrEventApp.
            ?lrChangeDis a addr:Change; addr:isChangeType ctype:LandmarkRelationDisappearance; addr:appliedTo ?landmarkRelation; addr:dependsOn ?lrEventDis.
            ?lrEventApp a addr:Event; addr:hasTime ?timeInstantApp.
            ?lrEventDis a addr:Event; addr:hasTime ?timeInstantDis.
            ?timeInstantApp a ?timeInstantAppType; addr:timeStamp ?timeInstantAppStamp; addr:timePrecision ?timeInstantAppPrec; addr:timeCalendar ?timeInstantAppCal.
            ?timeInstantDis a ?timeInstantDisType; addr:timeStamp ?timeInstantDisStamp; addr:timePrecision ?timeInstantDisPrec; addr:timeCalendar ?timeInstantDisCal.
        }}
    }}
    WHERE {{
        BIND({factoids_graph_uri.n3()} AS ?gf)
        BIND({tmp_graph_uri.n3()} AS ?gt)
        BIND(URI(CONCAT(STR(URI(wikidata:)), "LR_", STRUUID())) AS ?landmarkRelation)
        BIND(URI(CONCAT(STR(URI(wikidata:)), "CHA_", STRUUID())) AS ?lrChangeApp)
        BIND(URI(CONCAT(STR(URI(wikidata:)), "CHD_", STRUUID())) AS ?lrChangeDis)
        BIND(URI(CONCAT(STR(URI(wikidata:)), "EVA_", STRUUID())) AS ?lrEventApp)
        BIND(URI(CONCAT(STR(URI(wikidata:)), "EVD_", STRUUID())) AS ?lrEventDis)
        BIND(IF(BOUND(?timeInstantAppStamp), URI(CONCAT(STR(URI(wikidata:)), "TIA_", STRUUID())), ?x) AS ?timeInstantApp)
        BIND(IF(BOUND(?timeInstantDisStamp), URI(CONCAT(STR(URI(wikidata:)), "TID_", STRUUID())), ?x) AS ?timeInstantDis)
        {{
            SELECT DISTINCT * WHERE {{
                ?lr a addr:LandmarkRelation; addr:isLandmarkRelationType ?landmarkRelationType; addr:locatum ?l; addr:relatum ?r.
                ?changeApp a addr:Change; addr:isChangeType ctype:LandmarkRelationAppearance; addr:appliedTo ?lr; addr:dependsOn [a addr:Event; addr:hasTime ?timeInstantAppRaw].
                ?changeDis a addr:Change; addr:isChangeType ctype:LandmarkRelationDisappearance; addr:appliedTo ?lr; addr:dependsOn [a addr:Event; addr:hasTime ?timeInstantDisRaw].
                OPTIONAL {{
                    ?lr prov:wasDerivedFrom ?prov.
                }}
                OPTIONAL {{
                    GRAPH ?gt {{?timeInstantAppRaw a ?timeInstantAppType}}
                    ?timeInstantAppType rdfs:subClassOf* addr:TemporalEntity.
                    ?timeInstantAppRaw addr:timeStamp ?timeInstantAppStamp; addr:timePrecision ?timeInstantAppPrec; addr:timeCalendar ?timeInstantAppCal.
                    }}
                OPTIONAL {{
                    GRAPH ?gt {{?timeInstantDisRaw a ?timeInstantDisType}}
                    ?timeInstantDisType rdfs:subClassOf* addr:TemporalEntity.
                    ?timeInstantDisRaw addr:timeStamp ?timeInstantDisStamp; addr:timePrecision ?timeInstantDisPrec; addr:timeCalendar ?timeInstantDisCal.
                    }}
                ?cml skos:closeMatch ?l.
                ?cmr skos:closeMatch ?r.
            }}
        }}
    }}
    """

    gd.update_query(query, graphdb_url, repository_name)
    gd.remove_graph_from_uri(tmp_graph_uri)

#### Définition d'un processus de création des données de Wikidata

In [ ]:
def get_data_from_wikidata(wdpt_csv_file, wdpa_csv_file, wdpl_csv_file):
    """
    Obtenir les fichiers CSV pour les données provenant de Wikidata
    """
    get_paris_thoroughfares_from_wikidata(wdpt_csv_file)
    get_areas_around_paris_from_wikidata(wdpa_csv_file)
    get_paris_locations_from_wikidata(wdpl_csv_file)


def create_factoid_process_wikidata(graphdb_url, repository_name, namespace_prefixes, tmp_folder,
                                    ontorefine_url, ontorefine_cmd, 
                                    ont_file, ontology_named_graph_name,
                                    factoids_graph_name, permanent_graph_name,
                                    wdpt_csv_file, wdpt_mapping_file, wdpt_kg_file,
                                    wdpa_csv_file, wdpa_mapping_file, wdpa_kg_file,
                                    wdpl_csv_file, wdpl_mapping_file, wdpl_kg_file):
    """
    Fonction pour faire l'ensemble des processus relatifs à la création des factoïdes pour les données de Wikidata
    """

    # Création d'un répertoire des factoïdes pour les données de Wikidata
    create_factoid_repository(graphdb_url, repository_name, namespace_prefixes, tmp_folder, ont_file, ontology_named_graph_name, disable_same_as=False, clear_if_exists=True)

    # Récupération des URI des graphes nommés
    factoids_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, repository_name, factoids_graph_name))
    permanent_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, repository_name, permanent_graph_name))
    tmp_graph_name = "tmp"
    tmp_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, repository_name, tmp_graph_name))

    # À partir des fichiers csv décrivant les repères liés à Paris (voies et zones), convertir en un graphe de connaissance selon le mapping défini
    # Puis import du graphe dans le répertoire dont le nom est `repository_name` et dans le graphe nommé donné par `graph_name`
    from_raw_to_data_to_graphdb(graphdb_url, ontorefine_url, ontorefine_cmd, repository_name, factoids_graph_name, wdpt_csv_file, wdpt_mapping_file, wdpt_kg_file)
    from_raw_to_data_to_graphdb(graphdb_url, ontorefine_url, ontorefine_cmd, repository_name, factoids_graph_name, wdpa_csv_file, wdpa_mapping_file, wdpa_kg_file)
    from_raw_to_data_to_graphdb(graphdb_url, ontorefine_url, ontorefine_cmd, repository_name, tmp_graph_name, wdpl_csv_file, wdpl_mapping_file, wdpl_kg_file)

    # Transférer les relations spatiales entre les entités de Wikidata vers les entités créées
    transfert_landmark_relations_to_thoroughfares_wikidata(graphdb_url, repository_name, factoids_graph_uri, tmp_graph_uri)

    # Suppression des instants qui n'ont aucun timeStamp (instants sans date)
    remove_time_instant_without_timestamp(graphdb_url, repository_name, factoids_graph_uri)

    # Nettoyage du graphe
    clean_wikidata_graph(graphdb_url, repository_name, factoids_graph_uri, permanent_graph_uri)

    # Transfert de triplets non modifiables vers le graphe nommé permanent
    transfert_immutable_triples(graphdb_url, repository_name, factoids_graph_uri, permanent_graph_uri)
    
    # L'URI ci-dessous définit la source liée à Wikidata
    wdpt_source_uri = URIRef("http://rdf.geohistoricaldata.org/id/address/facts/Source_WD")
    create_source_wikidata(graphdb_url, repository_name, wdpt_source_uri, permanent_graph_uri)

    # Ajout de labels normalisés
    add_normalized_label_for_landmarks(graphdb_url, repository_name, factoids_graph_uri)
    
    # # Ajout de liens entre les ressources de type repère et la source
    # add_factoids_resources_links(graphdb_url, repository_name, wdpt_source_uri, factoids_graph_uri)

### OpenStreetMap (OSM)

#### Création des données liées à la source

In [ ]:
def create_source_osm(graphdb_url, repository_name, factoids_refactoids_uri:URIRef, facts_graph_uri:URIRef):
    query = f"""
        PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#> 
        PREFIX facts: <http://rdf.geohistoricaldata.org/id/address/facts/>
        PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>

        INSERT DATA {{
            GRAPH {facts_graph_uri.n3()} {{
                {factoids_refactoids_uri.n3()} a rico:Record;
                    rdfs:label "OpenStreetMap"@mul.
            }}
        }}
        """
    
    gd.update_query(query, graphdb_url, repository_name)

#### Nettoyage du graphe

Détection de repères décrits via plusieurs ressources et création d'une unique ressource pour chaque.

In [ ]:
# Détection des communes et arrondissements dupliqués et création d'une source centrale (?newLandmark) selon le code INSEE.
def clean_osm_graph(graphdb_url, repository_name, factoids_graph_uri, permanent_graph_uri):
    prefixes = """
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>
    PREFIX geofla: <http://data.ign.fr/def/geofla#>
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
    PREFIX ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/>
    """

    # Suppresion des numéros d'immeubles qui appartiennent à aucune voie
    query1 = prefixes + f"""
    DELETE {{
            ?hn ?p ?o.
            ?s ?p ?hn.
        }}
    WHERE {{
        ?hn a addr:Landmark; addr:isLandmarkType ltype:HouseNumber.
        MINUS {{?lr a addr:LandmarkRelation; addr:locatum ?hn; addr:relatum [a addr:Landmark; addr:isLandmarkType ?thoroughfare] }}
        {{?hn ?p ?o.}}UNION{{?s ?p ?hn.}}
    }}
    """
    
    queries = [query1]
    for query in queries:
        gd.update_query(query, graphdb_url, repository_name)

#### Définition d'un processus de création des données d'OSM

In [ ]:
def create_factoid_process_osm(graphdb_url, repository_name, namespace_prefixes, tmp_folder,
                               ontorefine_url, ontorefine_cmd, ont_file, ontology_named_graph_name,
                               factoids_graph_name, permanent_graph_name,
                               osm_csv_file, osm_mapping_file, osm_kg_file,
                               osm_hn_csv_file, osm_hn_mapping_file, osm_hn_kg_file):

    """
    Fonction pour faire l'ensemble des processus relatifs à la création des factoïdes pour les données d'OSM
    """

    # Création d'un répertoire des factoïdes pour les données d'OSM
    create_factoid_repository(graphdb_url, repository_name, namespace_prefixes, tmp_folder, ont_file, ontology_named_graph_name, disable_same_as=False, clear_if_exists=True)

    # Récupération des URI des graphes nommés
    factoids_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, repository_name, factoids_graph_name))
    permanent_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, repository_name, permanent_graph_name))

    # A partir des fichiers csv décrivant les adresses d'OSM dans Paris, convertir en un graphe de connaissance selon le mapping défini
    # Puis import du graphe dans le répertoire dont le nom est `repository_name` et dans le graphe nommé donné par `graph_name`
    from_raw_to_data_to_graphdb(graphdb_url, ontorefine_url, ontorefine_cmd, repository_name, factoids_graph_name, osm_csv_file, osm_mapping_file, osm_kg_file)
    from_raw_to_data_to_graphdb(graphdb_url, ontorefine_url, ontorefine_cmd, repository_name, factoids_graph_name, osm_hn_csv_file, osm_hn_mapping_file, osm_hn_kg_file)

    # Nettoyage du graphe
    clean_osm_graph(graphdb_url, repository_name, factoids_graph_uri, permanent_graph_uri)

    # Ajout d'éléments manquants
    add_missing_elements_for_landmarks(graphdb_url, repository_name, factoids_graph_uri)

    # L'URI ci-dessous définit la source liée à OSM
    osm_source_uri = URIRef("http://rdf.geohistoricaldata.org/id/address/facts/Source_OSM")
    create_source_osm(graphdb_url, repository_name, osm_source_uri, permanent_graph_uri)

    # Transfert de triplets non modifiables vers le graphe nommé permanent
    transfert_immutable_triples(graphdb_url, repository_name, factoids_graph_uri, permanent_graph_uri)

    # Ajout de labels normalisés
    add_normalized_label_for_landmarks(graphdb_url, repository_name, factoids_graph_uri)
    
    # Ajout de liens entre les ressources de type repère et la source
    add_factoids_resources_links(graphdb_url, repository_name, factoids_graph_uri)

# Création des faits

## Création de données dans le graphe des faits

### Définition de fonctions pour peupler le graphe des faits

In [ ]:
def create_unlinked_resources(graphdb_url, repository_name, refactoids_class:URIRef, refactoids_prefix:str, factoids_graph_uri:URIRef, facts_graph_uri:URIRef):
    # Créer des ressources en tant que fait et créer un lien de provenance
    query = f"""
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
    PREFIX facts: <http://rdf.geohistoricaldata.org/id/address/facts/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    INSERT {{
        GRAPH {factoids_graph_uri.n3()} {{
            ?resource owl:sameAs ?sourceResource.
        }}
        GRAPH {facts_graph_uri.n3()} {{
            ?resource a ?type.
        }}
    }}
    WHERE {{
        ?type rdfs:subClassOf* {refactoids_class.n3()}.
        GRAPH {factoids_graph_uri.n3()} {{
            ?sourceResource a ?type.
        }}
        MINUS {{?sourceResource owl:sameAs [a {refactoids_class.n3()}]}}
        BIND(URI(CONCAT(STR(URI(facts:)), "{refactoids_prefix}_", STRUUID())) AS ?resource)
    }}
    """
    gd.update_query(query, graphdb_url, repository_name)

def create_same_as_links_between_landmarks(graphdb_url, repository_name, factoids_graph_uri:URIRef, facts_graph_uri:URIRef):
    """
    Création de liens owl:sameAs entre des repères.
    """

    create_same_as_links_between_areas(graphdb_url, repository_name, factoids_graph_uri, facts_graph_uri)
    create_same_as_links_between_thoroughfares(graphdb_url, repository_name, factoids_graph_uri, facts_graph_uri)
    create_same_as_links_between_housenumbers(graphdb_url, repository_name, factoids_graph_uri, facts_graph_uri)

def create_same_as_links_between_areas(graphdb_url, repository_name, factoids_graph_uri:URIRef, facts_graph_uri:URIRef):
    """
    Pour les repères de type DISTRICT ou CITY définis dans le graphe nommé `factoids_graph_uri`, les lier avec un repère de même type défini dans `facts_graph_uri` s'ils ont un nom similaire.
    Le lien créé est mis dans `factoids_facts_graph_uri`.
    """
    prefixes = """
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    """

    query = prefixes + f"""
    INSERT {{
        GRAPH {factoids_graph_uri.n3()} {{
            ?factsLandmark owl:sameAs ?sourceLandmark.
        }}
    }}
    WHERE {{
        GRAPH {factoids_graph_uri.n3()} {{
            ?sourceLandmark a addr:Landmark; addr:isLandmarkType ?landmarkType; addr:hasAttribute ?sourceLandmarkAttr.
            ?sourceLandmarkAttr addr:isAttributeType ?attrType; addr:hasAttributeVersion [addr:versionValue ?versionValue].
        }}
        GRAPH {facts_graph_uri.n3()} {{
            ?factsLandmark a addr:Landmark; addr:isLandmarkType ?landmarkType; addr:hasAttribute ?factsLandmarkAttr.
            ?factsLandmarkAttr addr:isAttributeType ?attrType; addr:hasAttributeVersion [addr:versionValue ?versionValue].
           }}
        FILTER (?landmarkType IN (ltype:District, ltype:City, ltype:PostalCode))
        MINUS {{?factsLandmark owl:sameAs ?sourceLandmark}}
    }}
    """

    gd.update_query(query, graphdb_url, repository_name)

def create_same_as_links_between_thoroughfares(graphdb_url, repository_name, factoids_graph_uri:URIRef, facts_graph_uri:URIRef):
    """
    Pour les repères de type VOIE définis dans le graphe nommé `factoids_graph_uri`, les lier avec un repère de même type défini dans `facts_graph_uri` s'ils ont un nom similaire.
    Le lien créé est mis dans `factoids_facts_graph_uri`.
    """
    prefixes = """
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    """

    query = prefixes + f"""
    INSERT {{
        GRAPH {factoids_graph_uri.n3()} {{
            ?factsLandmark owl:sameAs ?sourceLandmark.
        }}
    }}
    WHERE {{
        GRAPH {factoids_graph_uri.n3()} {{
            ?sourceLandmark a addr:Landmark; addr:isLandmarkType ltype:Thoroughfare.
        }}
        GRAPH {facts_graph_uri.n3()} {{
            ?factsLandmark a addr:Landmark; addr:isLandmarkType ltype:Thoroughfare.
           }}
        MINUS {{?factsLandmark owl:sameAs ?sourceLandmark}}
        ?sourceLandmark skos:hiddenLabel ?label.
        ?factsLandmark skos:hiddenLabel ?label.
    }}
    """

    gd.update_query(query, graphdb_url, repository_name)


def create_same_as_links_between_housenumbers(graphdb_url, repository_name, factoids_graph_uri:URIRef, facts_graph_uri:URIRef):
    """
    Pour les repères de type HOUSENUMBER définis dans le graphe nommé `factoids_graph_uri`, les lier avec un repère de même type défini dans `facts_graph_uri` s'ils ont un nom similaire.
    Le lien créé est mis dans `factoids_facts_graph_uri`.
    """
    prefixes = """
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/>
    PREFIX lrtype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkRelationType/>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    """

    query = prefixes + f"""
    INSERT {{
        GRAPH {factoids_graph_uri.n3()} {{
            ?factsHN owl:sameAs ?sourceHN.
        }}
    }}
    WHERE {{
        BIND(ltype:HouseNumber AS ?landmarkType)
        BIND(lrtype:Along AS ?landmarkRelationType)
        GRAPH {factoids_graph_uri.n3()} {{
            ?sourceHN a addr:Landmark; addr:isLandmarkType ?landmarkType.
        }}
        GRAPH {facts_graph_uri.n3()} {{
            ?factsHN a addr:Landmark; addr:isLandmarkType ?landmarkType.
           }}

        ?sourceLR a addr:LandmarkRelation; addr:isLandmarkRelationType ?landmarkRelationType; addr:locatum ?sourceHN; addr:relatum ?thoroughfare.
        ?factsLR a addr:LandmarkRelation; addr:isLandmarkRelationType ?landmarkRelationType; addr:locatum ?factsHN; addr:relatum ?thoroughfare.
        MINUS {{?factsHN owl:sameAs ?sourceHN}}
        ?sourceHN skos:hiddenLabel ?label.
        ?factsHN skos:hiddenLabel ?label.
    }}
    """

    gd.update_query(query, graphdb_url, repository_name)

def create_same_as_links_between_landmark_relations(graphdb_url, repository_name, factoids_graph_uri:URIRef, facts_graph_uri:URIRef):
    """
    Pour des relations entre repères dans le graphe nommé `factoids_graph_uri`, les lier avec une relation entre repères dans `facts_graph_uri` qui sont similaires (mêmes locatum, relatums et type de relation).
    Le lien créé est mis dans `factoids_facts_graph_uri`.
    """

    query = f"""
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    INSERT {{
        GRAPH {factoids_graph_uri.n3()} {{ ?lr1 owl:sameAs ?lr2. }}
    }}
    WHERE {{
        GRAPH {facts_graph_uri.n3()} {{ ?lr1 a ?typeLR1. }}
        GRAPH {factoids_graph_uri.n3()} {{ ?lr2 a ?typeLR2. }}
        ?typeLR1 rdfs:subClassOf addr:LandmarkRelation.
        ?typeLR2 rdfs:subClassOf addr:LandmarkRelation.
        ?lr1 addr:isLandmarkRelationType ?lrt; addr:locatum ?l.
        ?lr2 addr:isLandmarkRelationType ?lrt; addr:locatum ?l.
        MINUS
        {{
            SELECT DISTINCT ?lr1 ?lr2 WHERE {{
                ?lr1 addr:isLandmarkRelationType ?lrt; addr:locatum ?l.
                ?lr2 addr:isLandmarkRelationType ?lrt; addr:locatum ?l.
                ?lr1 addr:relatum ?r.
                MINUS {{
                    ?lr2 addr:relatum ?r.
                }}
            }}
        }}
        MINUS
        {{
            SELECT DISTINCT ?lr1 ?lr2 WHERE {{
                ?lr1 addr:isLandmarkRelationType ?lrt; addr:locatum ?l.
                ?lr2 addr:isLandmarkRelationType ?lrt; addr:locatum ?l.
                ?lr2 addr:relatum ?r.
                MINUS {{
                    ?lr1 addr:relatum ?r.
                }}
            }}
        }}  
    }}
    """

    gd.update_query(query, graphdb_url, repository_name)

def store_interesting_implicit_triples(graphdb_url, repository_name, tmp_graph_uri:URIRef):
    query = f"""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
    PREFIX facts: <http://rdf.geohistoricaldata.org/id/address/facts/>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>
    PREFIX prov: <http://www.w3.org/ns/prov#>
    
    INSERT {{
        GRAPH {tmp_graph_uri.n3()} {{
            ?s ?p ?o.
        }}
    }}
    WHERE {{
        ?s ?p ?o.

        MINUS {{
            GRAPH ?g {{ ?s ?p ?o. }}
        }}
        FILTER(?p in (addr:isAttributeType,addr:isChangeType,addr:isLandmarkType,addr:isLandmarkRelationType,
            addr:hasTime,addr:timeCalendar,addr:timePrecision,addr:timeStamp,
            addr:hasAttribute,addr:hasAttributeVersion,addr:versionValue,addr:appliedTo,addr:dependsOn,addr:makesEffective,addr:outdates,
            addr:targets,addr:locatum,addr:relatum,addr:firstStep,addr:nextStep,
            rdfs:label,skos:hiddenLabel,skos:closeMatch,rico:isOrWasDescribedBy,prov:wasDerivedFrom
        ))
    }}
    """

    gd.update_query(query, graphdb_url, repository_name)

def get_facts_implicit_triples(graphdb_url, repository_name, factoids_graph_uri:URIRef, facts_graph_uri:URIRef, tmp_graph_uri:URIRef):
    """
    Tous les triplets intéressants (selon la propriété du triplet) ont été stockés dans un graphe nommé temporaire.
    Transférer les triplets dont :
    - les sujets sont des ressources RS définies dans le graphe des faits (il existe <RS a ?rtype> dans le graphe des faits) 
    - les objets ne sont pas des ressources RO définies dans le graphe des factoïdes (celles tel qu'il n'existe pas <RO a ?rtype> dans le graphe des factoïdes)
    """
    
    query = f"""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
    PREFIX facts: <http://rdf.geohistoricaldata.org/id/address/facts/>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>
    
    DELETE {{
        GRAPH ?gt {{
            ?s ?p ?o.
        }}
    }}
    INSERT {{
        GRAPH ?gf {{
            ?s ?p ?o.
        }}
    }}
    WHERE {{
        BIND ({facts_graph_uri.n3()} AS ?gf)
        BIND ({factoids_graph_uri.n3()} AS ?gs)
        BIND ({tmp_graph_uri.n3()} AS ?gt)

        GRAPH ?gt {{
            ?s ?p ?o.
        }}

        GRAPH ?gf {{
            ?s a ?sType.
        }}

        OPTIONAL {{
            GRAPH ?g {{?o a ?oType}}
        }}

        BIND(isIRI(?o) AS ?isIRI)
        BIND(IF(BOUND(?g) && ?g != ?gs, "true"^^xsd:boolean, "false"^^xsd:boolean) AS ?iriInFacts)
        FILTER(?isIRI = "false"^^xsd:boolean || ?iriInFacts = "true"^^xsd:boolean)
    }}
    """

    gd.update_query(query, graphdb_url, repository_name)

def transfer_implicit_triples(graphdb_url, repository_name, factoids_graph_uri:URIRef, facts_graph_uri:URIRef):
    tmp_graph_name = "tmp_named_graph"
    tmp_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, repository_name, tmp_graph_name))

    # On stocke dans un graphe nommé temporaire les triplets implicites qui sont intéressants (selon la propriété du triplet)
    store_interesting_implicit_triples(graphdb_url, repository_name, tmp_graph_uri)

    # Suppression des liens owl:sameAs pour casser les liens implicites qui ont été stockés explicitement dans le graphe nommé
    remove_all_same_as_triples(graphdb_url, repository_name)

    # Refaire les inférences pour supprimer notamment les liens owl:sameAs implicites qui ne sont pas supprimés
    gd.reinfer_repository(graphdb_url, repository_name)

    # Transférer les triplets stockés dans le graphe nommé temporaire qui ne sont pas liés aux factoïdes
    get_facts_implicit_triples(graphdb_url, repository_name, factoids_graph_uri, facts_graph_uri, tmp_graph_uri)
    
    # Supprimer tous les triples du graphe nommé temporaire
    gd.remove_graph(graphdb_url, repository_name, tmp_graph_name)


def remove_all_same_as_triples(graphdb_url, repository_name):
    query = """
    PREFIX owl: <http://www.w3.org/2002/07/owl#>

    DELETE {?s owl:sameAs ?o} WHERE {?s owl:sameAs ?o}
    """
    
    gd.update_query(query, graphdb_url, repository_name)

### Création de ressoures à partir des différentes sources

La création de voies se fait de la manière suivante :
* création de liens (avec `owl:sameAs`) entre les voies du graphe des faits et celles du graphe des factoïdes
* avec des règles d'inférence, des nouveaux liens `owl:sameAs` en sont déduits
* pour chaque ressource définie dans les factoïdes, on regarde si elle existe dans le graphe des faits (si elle est liée avec un `owl:sameAs` avec une ressource du graphe des faits)
* pour les ressources des factoïdes non liées, on créé son équivalent dans le graphe des faits

In [ ]:
def links_factoids_with_facts(graphdb_url, repository_name, factoids_graph_uri:URIRef, facts_graph_uri:URIRef):
    create_same_as_links_between_landmarks(graphdb_url, repository_name, factoids_graph_uri, facts_graph_uri)
    # TODO : corriger la fonction pour éviter des problèmes de propagation non voulues
    create_same_as_links_between_landmark_relations(graphdb_url, repository_name, factoids_graph_uri, facts_graph_uri)
    addr_ns = Namespace("http://rdf.geohistoricaldata.org/def/address#")
    # rico_ns = Namespace("https://www.ica.org/standards/RiC/ontology#")

    resource_classes = {"LM": addr_ns["Landmark"], "LR": addr_ns["LandmarkRelation"], "ADDR": addr_ns["Address"],
                        "ATTR": addr_ns["Attribute"], "AV":addr_ns["AttributeVersion"], "CG": addr_ns["Change"], "EV":addr_ns["Event"], "TE": addr_ns["TemporalEntity"]}
    for prefix, class_name in resource_classes.items():
        create_unlinked_resources(graphdb_url, repository_name, class_name, prefix, factoids_graph_uri, facts_graph_uri)

### Réinitialisation du répertoire

La réinitialisation du répertoire :
* export du graphe des faits dans un fichier TTL
* suppression de tous les triplets du répertoire (explicites comme implicites)
* réimport des fichiers TTL de l'ontologie et du graphe des faits

:TODO: voir si c'est possible de supprimer facilement les triplets implicites pour éviter de faire appel à cette fonction.

In [ ]:
def reload_repository(graphdb_url, repository_name, facts_ttl_file, facts_graph_name, ont_file, ontology_named_graph_name):
    facts_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, repository_name, facts_graph_name))

    # Exporter le graphe des faits dans un fichier TTL
    gd.export_data_from_repository(graphdb_url, repository_name, facts_ttl_file, facts_graph_uri)

    # Réinitialiser le répertoire et le remplir une nouvelle fois avec l'ontologie et le graphe des faits
    gd.clear_repository(graphdb_url, repository_name)
    gd.import_ttl_file_in_graphdb(graphdb_url, repository_name, ont_file, ontology_named_graph_name)
    gd.import_ttl_file_in_graphdb(graphdb_url, repository_name, facts_ttl_file, graph_name=facts_graph_name)

### Import des factoïdes dans le graphe des faits

L'import des factoïdes dans le graphe des faits se fait en trois étapes :
* liaison des éléments du graphe des factoides avec celui des faits (`links_factoids_with_facts()`)
* avec l'étape précédente, on a des inférences qui se font et `transfer_implicit_triples()` récupère les triplets implicites intéressants pour les expliciter en les mettant dans le graphe des faits
* `reload_repository()` exporte le graphe des faits dans un fichier temporaire afin de nettoyer le répertoire (suppression de triplets implicites inutiles), le graphe des faits est rechargé dans le répertoire.

In [ ]:
def import_factoids_in_facts(graphdb_url, repository_name, factoids_graph_name, facts_graph_name, facts_ttl_file):
    facts_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, repository_name, facts_graph_name))
    factoids_graph_uri = URIRef(gd.get_graph_uri_from_name(graphdb_url, repository_name, factoids_graph_name))

    links_factoids_with_facts(graphdb_url, repository_name, factoids_graph_uri, facts_graph_uri)
    transfer_implicit_triples(graphdb_url, repository_name, factoids_graph_uri, facts_graph_uri)
    reload_repository(graphdb_url, repository_name, facts_ttl_file, facts_graph_name, ont_file, ontology_named_graph_name)

## Processus final et itératif

### Création des factoïdes dans des répertoires

Pour chaque source, des factoïdes sont créés indépendamment dans des répertoires distincts

#### Ville de Paris


In [ ]:
# create_factoid_process_ville_paris(graphdb_url, vpt_repository_name, namespace_prefixes, tmp_folder,
#                                    ontorefine_url, ontorefine_cmd, ont_file, ontology_named_graph_name, factoids_graph_name, permanent_graph_name,
#                                    vpta_csv_file, vpta_mod_csv_file, vpta_csv_file_delimiter, vpta_csv_file_geom_col_name,
#                                    vptc_csv_file, vpta_mapping_file, vptc_mapping_file, vpta_kg_file, vptc_kg_file)

####  BAN


In [ ]:
# create_factoid_process_ban(graphdb_url, bpa_repository_name, namespace_prefixes, tmp_folder,
#                            ontorefine_url, ontorefine_cmd,
#                            ont_file, ontology_named_graph_name, factoids_graph_name, permanent_graph_name,
#                            bpa_csv_file, bpa_mapping_file, bpa_kg_file)

#### Wikidata


In [ ]:
# # get_data_from_wikidata(wdpt_csv_file, wdpa_csv_file, wdpl_csv_file)
# create_factoid_process_wikidata(graphdb_url, wd_repository_name, namespace_prefixes, tmp_folder, 
#                                 ontorefine_url, ontorefine_cmd, 
#                                 ont_file, ontology_named_graph_name, factoids_graph_name, permanent_graph_name,
#                                 wdpt_csv_file, wdpt_mapping_file, wdpt_kg_file, 
#                                 wdpa_csv_file, wdpa_mapping_file, wdpa_kg_file,
#                                 wdpl_csv_file, wdpl_mapping_file, wdpl_kg_file)

#### OSM

In [ ]:
# create_factoid_process_osm(graphdb_url, osm_repository_name, namespace_prefixes, tmp_folder,
#                         ontorefine_url, ontorefine_cmd, ont_file, ontology_named_graph_name, factoids_graph_name, permanent_graph_name,
#                         osm_csv_file, osm_mapping_file, osm_kg_file,
#                         osm_hn_csv_file, osm_hn_mapping_file, osm_hn_kg_file)

### Insertion des factoïdes dans le graphe des faits

#### Ville de Paris

In [ ]:
transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, vpt_repository_name, vpt_factoids_kg_file, vpt_permanent_kg_file, factoids_graph_name, facts_graph_name, permanent_graph_name)
import_factoids_in_facts(graphdb_url, facts_repository_name, factoids_graph_name, facts_graph_name, facts_ttl_file)

#### Wikidata

In [ ]:
transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, wd_repository_name, wd_factoids_kg_file, wd_permanent_kg_file, factoids_graph_name, facts_graph_name, permanent_graph_name)
import_factoids_in_facts(graphdb_url, facts_repository_name, factoids_graph_name, facts_graph_name, facts_ttl_file)

#### BAN

In [ ]:
transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, bpa_repository_name, bpa_factoids_kg_file, bpa_permanent_kg_file, factoids_graph_name, facts_graph_name, permanent_graph_name)
import_factoids_in_facts(graphdb_url, facts_repository_name, factoids_graph_name, facts_graph_name, facts_ttl_file)

#### OSM

In [ ]:
transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, osm_repository_name, osm_factoids_kg_file, osm_permanent_kg_file, factoids_graph_name, facts_graph_name, permanent_graph_name)
import_factoids_in_facts(graphdb_url, facts_repository_name, factoids_graph_name, facts_graph_name, facts_ttl_file)